In [106]:
from proj1_helpers import *
from implementations import *
from auxiliary import *
import pickle
import matplotlib.pyplot as plt
import datetime
import random
import copy

%matplotlib inline
%reload_ext autoreload
%autoreload 2

## Data loading


### Training sets

In [2]:
easy_data_set = False # changer pour sélectionner un set de dim (1000,2) ou le vrai set
data_subset = True # the data is divided in 8 subsets

if data_subset:
    r=8
else:
    r=1

# declare all variables as lists
ids=[0]*r
y=[0]*r
X=[0]*r
x=[0]*r
tx=[0]*r
mean_x=[0]*r
std_x=[0]*r
N=[0]*r
D=[0]*r
prop=[0]*r

if easy_data_set:
    ids[0], y[0], X[0] = load_easy_data(sub_sample=False)
else:
    ids[0], y[0], X[0] = load_csv_data('../data/train.csv')
    len_tot = X[0].shape[0]
    if data_subset:
        ids, y, X = split_data_boson(ids[0], X[0], y[0])

    else:
        # Only 100% clean data
        ranges = [(1,4), (7,12), (13,23), (29,30)]
        keep_idx = build_idx(ranges)
        X[0] = X[0][:,keep_idx]


for r in range(len(X)):
    x[r], mean_x[r], std_x[r] = standardize(X[r])    # standardize
    clean_data = [x for x in x[r] if not -999. in x] # check tidyness of data
    print(len(clean_data)/len(x[r])*100) 
    tx[r] = build_poly(x[r], 1)                      # add column of 1's
    N[r] = x[r].shape[0]
    D[r] = x[r].shape[1]
    prop[r] = len(x[r])/len_tot
    print(prop[r])


100.0
0.104492
100.0
0.29516
100.0
0.030248
100.0
0.279928
100.0
0.011808
100.0
0.189708
100.0
0.005908
100.0
0.082748


### Testing sets

In [3]:
if data_subset:
    r=8
else:
    r=1

# declare variables
ids_ukn=[0]*r
y_ukn=[0]*r
X_ukn=[0]*r
x_ukn=[0]*r
tx_ukn=[0]*r
mean_x_ukn=[0]*r
std_x_ukn=[0]*r


# takes time, run only when needed
ids_ukn[0], y_ukn[0], X_ukn[0] = load_csv_data('../data/test.csv')

if data_subset:
    ids_ukn, y_ukn, X_ukn = split_data_boson(ids_ukn[0], X_ukn[0], y_ukn[0])
else:
    ranges = [(1,4), (7,12), (13,23), (29,30)]
    keep_idx = build_idx(ranges)
    X_ukn[0] = X_ukn[0][:,keep_idx]


for r in range(len(X_ukn)):
    x_ukn[r], mean_x_ukn[r], std_x_ukn[r] = standardize(X_ukn[r])    # standardize
    clean_data = [x for x in x_ukn[r] if not -999. in x]     # check tidyness of data
    print(len(clean_data)/len(x_ukn[r])*100) 
    tx_ukn[r] = build_poly(x_ukn[r], 1)                      # add column of 1's

100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0


Dividing data in 8 subsets

# Training
Différents types de training possibles. Normalement, seul le logistic regression a du sens. Les autres sont là pour pouvoir tester les diverses fonctions à fournir dans `implementation.py`.

In [14]:
if data_subset:
    r=8
else:
    r=1

loss=[0]*r
w=[0]*r
w_LS=[0]*r
w_LS_GD=[0]*r
w_LS_SGD=[0]*r
w_LS_degree=[0]*r
w_ridge_reg=[0]*r
w_LR=[0]*r

ratio_error_train=[0]*r
ratio_error_test=[0]*r


## Least squares

In [5]:
for r in range(len(x)):
    tx[r] = build_poly(x[r], 1)
    w_LS[r], loss[r] = least_squares(y[r],tx[r])
print(loss)

[0.088128177574748584]


## Least squares (GD)

In [33]:
# w_initial = w_LS

max_iters = 100
gammas = [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.2, 0.5]
losses = []
ws = [] 

for gamma in gammas:
    for r in range(len(x)):
        w_initial = np.array([0]*(D[r]+1))
        w[r], loss[r] = least_squares_GD(y[r], tx[r], w_initial, max_iters, gamma)
    losses.append(reduce_size_of_loss(loss))
    ws.append(w)

In [34]:
idx = np.argmin(losses) # get best experiment
w_LS_GD = ws[idx]

print("Take experiment {i} out of {tot} experiments, loss={l}, gamma={ga}, max_iter={mi}".format(
           i=idx+1, tot=len(ws), l=losses[idx], ga=gammas[idx],mi=max_iters))


Take experiment 10 out of 11 experiments, loss=0.08817998222515805, gamma=0.2, max_iter=100


## Least squares (SGD)

In [35]:
max_iters = 500
gammas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.2, 0.4, 0.5]
batch_size = 10
losses = []
ws = []

for gamma in gammas:
    for r in range(len(x)):
        w_initial = np.array([0]*(D[r]+1))
        w[r], loss[r] = least_squares_SGD(y[r], tx[r], w_initial, batch_size, max_iters, gamma)
    losses.append(reduce_size_of_loss(loss))
    ws.append(w)

In [36]:
idx = np.argmin(losses) # get best experiment
w_LS_SGD = ws[idx]

print("Take experiment {i} out of {tot} experiments, loss={l}, gamma={g}".format(
           i=idx+1, tot=len(ws), l=losses[idx], g=gammas[idx]))

print(losses)
print(len(ws))
w_LS_SGD = ws[7]

Take experiment 4 out of 8 experiments, loss=0.0900408552945142, gamma=0.01
[0.17031631432450312, 0.16180834781038447, 0.11663324048589102, 0.090040855294514194, 0.10213861307346044, 0.14938097611143059, 9.9741316832985045e+44, 1.6072455085816073e+107]
8


## Polynomial regression

In [50]:
"""Constructing the polynomial basis function expansion of the data,
   and then running least squares regression."""
# define parameters
degrees = [1, 3, 7, 10, 11, 12, 13, 14, 15]
losses = []
ws = []
ratio_errs = []
deg = []
    
phi = build_poly(x, 2)  # form the data to do polynomial regression.
    
for ind, degree in enumerate(degrees):
    phi = build_poly(x, degree)  # form the data to do polynomial regression.
    # least square and calculate RMSE
        
    w, loss_degree_LS = least_squares(y, phi)
    error, ratio_error = compute_classification_error(y, phi, w, logistic_reg=False)
    
    ratio_errs.append(ratio_error)
    
    losses.append(loss_degree_LS)
    ws.append(w)
    deg.append(degree)
    print("Processing {i}th experiment, degree={d}, loss={loss:.3f}, err_ratio={err:.3f}".format(
           i=ind, d=degree, loss=loss_degree_LS, err=ratio_error))


Processing 0th experiment, degree=1, loss=0.088, err_ratio=0.266
Processing 1th experiment, degree=3, loss=0.082, err_ratio=0.240
Processing 2th experiment, degree=7, loss=0.079, err_ratio=0.225
Processing 3th experiment, degree=10, loss=0.076, err_ratio=0.210
Processing 4th experiment, degree=11, loss=0.075, err_ratio=0.206
Processing 5th experiment, degree=12, loss=0.075, err_ratio=0.205
Processing 6th experiment, degree=13, loss=0.075, err_ratio=0.205
Processing 7th experiment, degree=14, loss=0.136, err_ratio=0.293
Processing 8th experiment, degree=15, loss=0.080, err_ratio=0.219


In [9]:
idx = np.argmin(ratio_errs) # get best experiment
w_degree_LS = ws[idx]


print("Take experiment {i} out of {tot} experiments, degree={d}, err_ratio={err}".format(
           i=idx, tot=len(ws), d=deg[idx], err=ratio_errs[idx]))


Take experiment 6 out of 9 experiments, degree=13, err_ratio=0.204712


## Ridge Regression
To do with subsets

In [6]:
lambdas = np.logspace(-5, 2, 15)

ratio = 0.9
degrees = [1, 3, 7, 12]

# split the data, and return train and test data
x_train, x_test, y_train, y_test = split_data(x, y, ratio)
    
    
loss_tr = []
loss_te = []
ratio_err_trains = []
ratio_err_tests = []
ws = []
deg=[]

for degree in degrees:
    # form train and test data with polynomial basis function
    phi_train = build_poly(x_train, degree)
    phi_test = build_poly(x_test, degree)
    for ind, lambda_ in enumerate(lambdas):
        # ridge regression with a given lambda
        w, loss = ridge_regression(y_train, phi_train, lambda_)
        ws.append(w)
        loss_tr.append(compute_loss(y_train,phi_train,w))
        loss_te.append(compute_loss(y_test,phi_test,w))
    
        error, ratio_error_train = compute_classification_error(y_train, phi_train, w, logistic_reg=False)
        error, ratio_error_test = compute_classification_error(y_test, phi_test, w, logistic_reg=False)
    
        ratio_err_trains.append(ratio_error_train)
        ratio_err_tests.append(ratio_error_test)
        deg.append(degree)

        print("proportion={p}, degree={d}, lambda={l:.3f}, Train loss,err_ratio=({tr:.3f},{err_ratio_tr:.3f}), Test loss,err_ratio=({te:.3f},{err_ratio_te:.3f})".format(
            p=ratio, d=degree, l=lambda_ ,tr=loss_tr[ind], te=loss_te[ind], err_ratio_tr=ratio_error_train, err_ratio_te=ratio_error_test))

proportion=0.9, degree=1, lambda=0.000, Train loss,err_ratio=(0.088,0.266), Test loss,err_ratio=(0.089,0.269)
proportion=0.9, degree=1, lambda=0.000, Train loss,err_ratio=(0.088,0.266), Test loss,err_ratio=(0.089,0.269)
proportion=0.9, degree=1, lambda=0.000, Train loss,err_ratio=(0.088,0.266), Test loss,err_ratio=(0.089,0.269)
proportion=0.9, degree=1, lambda=0.000, Train loss,err_ratio=(0.088,0.266), Test loss,err_ratio=(0.089,0.270)
proportion=0.9, degree=1, lambda=0.001, Train loss,err_ratio=(0.088,0.266), Test loss,err_ratio=(0.089,0.270)
proportion=0.9, degree=1, lambda=0.003, Train loss,err_ratio=(0.088,0.266), Test loss,err_ratio=(0.089,0.270)
proportion=0.9, degree=1, lambda=0.010, Train loss,err_ratio=(0.088,0.268), Test loss,err_ratio=(0.089,0.272)
proportion=0.9, degree=1, lambda=0.032, Train loss,err_ratio=(0.089,0.275), Test loss,err_ratio=(0.090,0.277)
proportion=0.9, degree=1, lambda=0.100, Train loss,err_ratio=(0.092,0.299), Test loss,err_ratio=(0.092,0.300)
proportion

In [7]:
idx = np.argmin(ratio_err_tests) # get best experiment
w_ridge_reg = ws[idx]


print("Take experiment {i} out of {tot} experiments, degree={d}, err_ratio={err}".format(
           i=idx, tot=len(ws), d=deg[idx], err=ratio_err_tests[idx]))

Take experiment 49 out of 60 experiments, degree=12, err_ratio=0.20796


## Logistic regression

In [107]:
max_iters = 500
gammas = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
#gammas = [1e-5, 1e-4]
losses = np.zeros([len(gammas), len(x)])
errors = np.zeros([len(gammas)])
ws = []
degree = 3

for r in range(len(x)):
    tx[r] = build_poly(x[r], degree)

for idx_g, gamma in enumerate(gammas):
    print("Run experiment {i}/{tot}".format(i=idx_g+1, tot=len(gammas)))
    for r in range(len(x)):
        w_initial = np.array([0]*(tx[r].shape[1]))
        w[r], loss[r] = logistic_regression(y[r], tx[r], w_initial, max_iters, gamma)
        nb_errors, error_mean = compute_classification_error(y[r], tx[r], w[r], logistic_reg=True)
        errors[idx_g] += error_mean*prop[r]
    losses[idx_g, :] = loss[:]  
    ws.append(copy.copy(w))
print("done")

Run experiment 1/8
Run experiment 2/8
Run experiment 3/8
Run experiment 4/8
Run experiment 5/8
Run experiment 6/8
Run experiment 7/8
Run experiment 8/8
done


In [108]:
idx = np.argmin(errors) # get best experiment

w_LR = ws[idx]

print("Take experiments {i} out of {tot} experiments, loss={l}, error={er:.3f}, gamma={g}".format(
           i=idx+1, tot=len(ws), l=losses_red[idx], er=errors[idx], g=gammas[idx]))


Take experiments 3 out of 8 experiments, loss=60205.77158984603, error=0.194, gamma=1e-06


# Evaluate for test set
Sert à afficher le nombre d'erreurs

In [111]:
final_error_mean = 0.0
for r in range(len(x)):
    tx[r] = build_poly(x[r], 3)
    w_selected = w_LR[r]
    nb_errors, error_mean = compute_classification_error(y[r], tx[r], w_selected, logistic_reg=True)
    #print(nb_errors)
    final_error_mean += error_mean*prop[r]
print(final_error_mean)

0.194076


In [24]:
print(w_LS)
print(w_LS_GD)
print(w_LS_SGD)
print(w_degree_LS)
print(w_ridge_reg)
print(w_LR)
print(w_LR_CV)

NameError: name 'w_LS' is not defined

## Cross validation with least squares
To do with subsets

In [43]:
seed = 6
degrees = [1, 3, 7, 11, 12, 13, 14]
exp = 0

loss_tr = []
loss_te = []
ratio_err_trains = []
ratio_err_tests = []
ws = []
deg=[]

k_fold = 4
k_indices = build_k_indices(y, k_fold, seed)

print(k_indices.shape)

for degree in degrees:
    phi = build_poly(x, degree)
    loss_tr_k_fold = []
    loss_te_k_fold = []
    ratio_err_trains_k_fold = []
    ratio_err_tests_k_fold = []
    ws_k_fold = []
    for k in range(k_fold):
        # split the data, and return train and test data: TODO
        phi_train, phi_test, y_train, y_test = split_data_k_fold(phi, y, k_indices, k)
            
        # calcualte weight through least square.: TODO
        w,loss = least_squares(y_train, phi_train)
        ws_k_fold.append(w)

        # calculate RMSE for train and test data,
        # and store them in rmse_tr and rmse_te respectively: TODO
        loss_tr_k_fold.append(compute_loss(y_train,phi_train,w), logistic_reg=False)
        loss_te_k_fold.append(compute_loss(y_test,phi_test,w), logistic_reg=False)
    
        error, ratio_error_train = compute_classification_error(y_train, phi_train, w)
        error, ratio_error_test = compute_classification_error(y_test, phi_test, w)
        
        ratio_err_trains_k_fold.append(ratio_error_train)
        ratio_err_tests_k_fold.append(ratio_error_test)
        
    get_best = False # get best or get mean
        
    if get_best:
        idx = np.argmin(ratio_err_tests_k_fold) # get best experiment
        ws.append(ws_k_fold[idx])
        loss_tr.append(loss_tr_k_fold[idx])
        loss_te.append(loss_te_k_fold[idx])
        ratio_err_trains.append(ratio_err_trains_k_fold[idx])
        ratio_err_tests.append(ratio_err_tests_k_fold[idx])
    else:
        ws.append(sum(ws_k_fold)/len(ws_k_fold))
        loss_tr.append(sum(loss_tr_k_fold)/len(loss_tr_k_fold))
        loss_te.append(sum(loss_te_k_fold)/len(loss_te_k_fold))
        ratio_err_trains.append(sum(ratio_err_trains_k_fold)/len(ratio_err_trains_k_fold))
        ratio_err_tests.append(sum(ratio_err_tests_k_fold)/len(ratio_err_tests_k_fold))
        
    deg.append(degree)
        
    print("exp={e}, degree={d}, Train loss, err_ratio=({tr:.3f},{er_tr:.3f}), Test loss, err_ratio=({te:.3f},{er_te:.3f})".format(
        e=exp, d=degree, tr=loss_tr[exp], te=loss_te[exp], er_tr=ratio_err_trains[exp], er_te=ratio_err_tests[exp]))
    exp += 1


(4, 62500)
exp=0, degree=1, Train loss, err_ratio=(0.088,0.266), Test loss, err_ratio=(0.088,0.266)
exp=1, degree=3, Train loss, err_ratio=(0.081,0.239), Test loss, err_ratio=(1.627,0.240)
exp=2, degree=7, Train loss, err_ratio=(0.079,0.225), Test loss, err_ratio=(82447866.810,0.225)
exp=3, degree=11, Train loss, err_ratio=(0.075,0.206), Test loss, err_ratio=(355213321008866394112.000,0.207)
exp=4, degree=12, Train loss, err_ratio=(0.075,0.205), Test loss, err_ratio=(4204608108250730594304.000,0.206)
exp=5, degree=13, Train loss, err_ratio=(0.077,0.210), Test loss, err_ratio=(24543286522935610108280832.000,0.211)
exp=6, degree=14, Train loss, err_ratio=(0.093,0.232), Test loss, err_ratio=(82609134220373679177004482560.000,0.232)


In [44]:
idx = np.argmin(ratio_err_tests) # get best experiment
w_LS_degree_CV = ws[idx]


print("Take experiment {i} out of {tot} experiments, degree={d}, err_ratio={err}".format(
           i=idx, tot=len(ws), d=deg[idx], err=ratio_err_tests[idx]))

Take experiment 4 out of 7 experiments, degree=12, err_ratio=0.205836


## Cross validation with logistic regression

In [20]:
def logistic_regression_split(x, y, degree, ratio, gamma, seed):
    """polynomial regression with different split ratios and different degrees."""
    # split the data, and return train and test data: TODO
    x_train, x_test, y_train, y_test = split_data(x, y, ratio, seed)

    # form train and test data with polynomial basis function: TODO
    phi_train = build_poly(x_train, degree)
    phi_test = build_poly(x_test, degree)
    
    w_initial = np.array([0]*(phi_train.shape[1]))
    max_iters = 1000

    
    # calcualte weight through least square.: TODO
    w,loss_tr = logistic_regression(y_train, phi_train, w_initial, max_iters, gamma)
    
    loss_te = compute_logistic_loss(y_test, phi_test, w)
    # compute error and loss for train and test data
    error, ratio_error_train = compute_classification_error(y_train, phi_train, w)
    error, ratio_error_test = compute_classification_error(y_test, phi_test, w)

    
    #print("proportion={p}, degree={d}, gamma={g}, Train loss, err_ratio=({tr:.3f},{er_tr:.3f}), Test loss, err_ratio=({te:.3f},{er_te:.3f})".format(
    #      p=ratio, d=degree, g=gamma, tr=loss_tr, te=loss_te, er_tr=ratio_error_train, er_te=ratio_error_test))
    return w, ratio_error_train, ratio_error_test


In [110]:
seed = 6
degrees = [1, 3, 7, 12]
split_ratios = [0.9, 0.5]
gammas = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]

degrees = [3]
split_ratios = [0.5]
gammas = [1e-7, 1e-6, 1e-5]

ws = []
ratio_err_trains = []
ratio_err_tests = []

print("There will be {exp} experiments".format(exp=len(degrees)*len(split_ratios)*len(gammas)))

for idx0, split_ratio in enumerate(split_ratios):
    for idx1, degree in enumerate(degrees):
        for idx2, gamma in enumerate(gammas):
            for r in range(len(x)):
                w[r], ratio_error_train[r], ratio_error_test[r] = logistic_regression_split(x[r], y[r], degree, split_ratio, gamma, seed)
            ws.append(copy.copy(w))
            ratio_err_trains.append(0)
            ratio_err_tests.append(0)
            for r in range(len(ratio_error_train)):
                ratio_err_trains[-1]+= ratio_error_train[r]*prop[r]
                ratio_err_tests[-1] += ratio_error_test[r]*prop[r]
            print("{exp}: proportion={p}, degree={d}, gamma={g}, Train err_ratio={er_tr:.3f}, Test err_ratio={er_te:.3f}".format(
                   exp=idx0*len(split_ratios)*len(degrees)*len(gammas)+idx1*len(degrees)*len(gammas)+idx2, p=split_ratio, d=degree, g=gamma, 
                   er_tr=ratio_err_trains[-1], er_te=ratio_err_tests[-1]))
print("done")

There will be 3 experiments
0: proportion=0.5, degree=3, gamma=1e-07, Train err_ratio=0.214, Test err_ratio=0.213
1: proportion=0.5, degree=3, gamma=1e-06, Train err_ratio=0.189, Test err_ratio=0.190
2: proportion=0.5, degree=3, gamma=1e-05, Train err_ratio=0.194, Test err_ratio=0.198
done


In [89]:
print(len(ratio_err_tests))
print(len(ws))

idx = np.argmin(ratio_err_tests) # get best experiment
w_LR_CV = ws[idx]


idx_split = int(idx/(len(degrees)*len(gammas)))
idx_degree_gamma = idx%(len(degrees)*len(gammas))
idx_degree = int(idx_degree_gamma/len(gammas))
idx_gamma = (idx_degree_gamma%len(gammas))


print("Take experiments {i} out of {tot} experiments, error tr={er_tr:.3f}, error test={er_te:.3f}, split_ratio={spl}, degree={deg}, gamma={g}".format(
           i=idx, tot=len(ws), er_tr=ratio_err_trains[idx], er_te=ratio_err_tests[idx], 
           spl=split_ratios[idx_split],  deg=degrees[idx_degree], g=gammas[idx_gamma]))


3
3
Take experiments 1 out of 3 experiments, error tr=0.190, error test=0.194, split_ratio=0.5, degree=3, gamma=1e-06


## Cross validation with regularized logistic regression
To do with subsets

In [5]:
def reg_logistic_regression_split(x, y, degree, ratio, gamma, lambda_, seed):
    """polynomial regression with different split ratios and different degrees."""
    # split the data, and return train and test data: TODO
    x_train, x_test, y_train, y_test = split_data(x, y, ratio, seed)

    # form train and test data with polynomial basis function: TODO
    phi_train = build_poly(x_train, degree)
    phi_test = build_poly(x_test, degree)
    
    w_initial = np.array([0]*(phi_train.shape[1]))
    max_iters = 1000

    
    # calcualte weight through logistic regression
    w, loss_tr = reg_logistic_regression(y_train, phi_train, lambda_, w_initial, max_iters, gamma, SGD=False)
    
    loss_te = compute_logistic_loss(y_test, phi_test, w)
    # compute error and loss for train and test data
    error, ratio_error_train = compute_classification_error(y_train, phi_train, w)
    error, ratio_error_test = compute_classification_error(y_test, phi_test, w)

    
    print("prop={p}, deg={d}, g={g:.3f}, l={l:.3f}, Train loss, err_ratio=({tr:.3f},{er_tr:.3f}), Test loss, err_ratio=({te:.3f},{er_te:.3f})".format(
          p=ratio, d=degree, g=gamma, l=lambda_, tr=loss_tr, te=loss_te, er_tr=ratio_error_train, er_te=ratio_error_test))
    return w, ratio_error_train, ratio_error_test


In [6]:
seed = 6
degrees = [1, 3, 7, 12]
split_ratios = [0.9, 0.8]
gammas = [1e-9, 1e-7, 1e-5, 1e-3, 1e-1]
lambdas = np.logspace(-5, 1, 15)

ws = []
ratio_err_trains = []
ratio_err_tests = []

print("There will be {exp} experiments".format(exp=len(degrees)*len(split_ratios)*len(gammas)))


for idx0, split_ratio in enumerate(split_ratios):
    for idx1, degree in enumerate(degrees):
        for idx2, gamma in enumerate(gammas):
            for idx3, lambda_ in enumerate(lambdas):
                for r in range(len(x)):
                    w[r], ratio_error_train[r], ratio_error_test[r] = reg_logistic_regression_split(x[r], y[r], degree, split_ratio, gamma, lambda_, seed)
                ws.append(w)
                ratio_err_trains.append(0)
                ratio_err_tests.append(0)
                for r in range(len(ratio_error_train)):
                    ratio_err_trains[-1]+= ratio_error_train[r]*prop[r]
                    ratio_err_tests[-1] += ratio_error_test[r]*prop[r]
                print("{exp}: proportion={p}, degree={d}, gamma={g}, Train err_ratio={er_tr:.3f}, Test err_ratio={er_te:.3f}".format(
                       exp=idx0*len(split_ratios)+idx1*len(degrees)+idx2*len(gammas)+idx3*len(lambdas)+1, p=split_ratio, d=degree, g=gamma, 
                       er_tr=ratio_err_trains[-1], er_te=ratio_err_tests[-1]))
            

prop=0.9, deg=1, g=0.000, l=0.000, Train loss, err_ratio=(150912.639,0.313), Test loss, err_ratio=(16748.710,0.308)
prop=0.9, deg=1, g=0.000, l=0.000, Train loss, err_ratio=(150883.074,0.313), Test loss, err_ratio=(16764.050,0.312)
prop=0.9, deg=1, g=0.000, l=0.000, Train loss, err_ratio=(150894.001,0.313), Test loss, err_ratio=(16757.850,0.312)
prop=0.9, deg=1, g=0.000, l=0.000, Train loss, err_ratio=(150886.808,0.313), Test loss, err_ratio=(16762.394,0.313)
prop=0.9, deg=1, g=0.000, l=0.001, Train loss, err_ratio=(150875.015,0.312), Test loss, err_ratio=(16768.347,0.315)
prop=0.9, deg=1, g=0.000, l=0.001, Train loss, err_ratio=(150886.975,0.312), Test loss, err_ratio=(16761.675,0.312)
prop=0.9, deg=1, g=0.000, l=0.004, Train loss, err_ratio=(150853.212,0.312), Test loss, err_ratio=(16778.842,0.317)
prop=0.9, deg=1, g=0.000, l=0.010, Train loss, err_ratio=(150908.928,0.313), Test loss, err_ratio=(16750.819,0.307)
prop=0.9, deg=1, g=0.000, l=0.027, Train loss, err_ratio=(150871.489,0.3

/Users/philippe/Desktop/ML_Projects/project1/phil/auxiliary.py:58: RuntimeWarning: overflow encountered in exp
  loss = np.sum(np.log(1.0+np.exp(tx.dot(w))) - y*(tx.dot(w)))
/Users/philippe/Desktop/ML_Projects/project1/phil/auxiliary.py:237: RuntimeWarning: invalid value encountered in double_scalars
  if abs(a-b) < threshold:


prop=0.9, deg=1, g=0.001, l=0.000, Train loss, err_ratio=(inf,0.303), Test loss, err_ratio=(643542.706,0.306)
prop=0.9, deg=1, g=0.001, l=0.000, Train loss, err_ratio=(inf,0.302), Test loss, err_ratio=(625869.052,0.301)
prop=0.9, deg=1, g=0.001, l=0.000, Train loss, err_ratio=(inf,0.299), Test loss, err_ratio=(609901.702,0.299)
prop=0.9, deg=1, g=0.001, l=0.000, Train loss, err_ratio=(inf,0.303), Test loss, err_ratio=(619249.526,0.301)
prop=0.9, deg=1, g=0.001, l=0.001, Train loss, err_ratio=(inf,0.302), Test loss, err_ratio=(631313.303,0.304)
prop=0.9, deg=1, g=0.001, l=0.001, Train loss, err_ratio=(inf,0.300), Test loss, err_ratio=(614549.961,0.301)
prop=0.9, deg=1, g=0.001, l=0.004, Train loss, err_ratio=(inf,0.302), Test loss, err_ratio=(609278.622,0.299)
prop=0.9, deg=1, g=0.001, l=0.010, Train loss, err_ratio=(inf,0.301), Test loss, err_ratio=(610910.813,0.297)
prop=0.9, deg=1, g=0.001, l=0.027, Train loss, err_ratio=(inf,0.304), Test loss, err_ratio=(660514.584,0.312)
prop=0.9, 

/Users/philippe/Desktop/ML_Projects/project1/phil/auxiliary.py:67: RuntimeWarning: overflow encountered in double_scalars
  loss_reg = lambda_ * w.T.dot(w)
/Users/philippe/Desktop/ML_Projects/project1/phil/auxiliary.py:58: RuntimeWarning: invalid value encountered in multiply
  loss = np.sum(np.log(1.0+np.exp(tx.dot(w))) - y*(tx.dot(w)))
/Users/philippe/Desktop/ML_Projects/project1/phil/auxiliary.py:58: RuntimeWarning: invalid value encountered in subtract
  loss = np.sum(np.log(1.0+np.exp(tx.dot(w))) - y*(tx.dot(w)))
/Users/philippe/Desktop/ML_Projects/project1/phil/auxiliary.py:50: RuntimeWarning: invalid value encountered in greater
  t[t>100.0]=100.0
/Users/philippe/Desktop/ML_Projects/project1/phil/auxiliary.py:51: RuntimeWarning: invalid value encountered in less
  t[t<-100.0]=-100.0
/Users/philippe/Desktop/ML_Projects/project1/phil/proj1_helpers.py:62: RuntimeWarning: invalid value encountered in less_equal
  y_pred[np.where(y_pred <= 0)] = 0
/Users/philippe/Desktop/ML_Projects/

prop=0.9, deg=1, g=0.500, l=3.728, Train loss, err_ratio=(nan,1.000), Test loss, err_ratio=(nan,1.000)


...

prop=0.9, deg=1, g=0.500, l=10.000, Train loss, err_ratio=(nan,1.000), Test loss, err_ratio=(nan,1.000)
prop=0.9, deg=3, g=0.000, l=0.000, Train loss, err_ratio=(inf,0.328), Test loss, err_ratio=(15455.236,0.327)
prop=0.9, deg=3, g=0.000, l=0.000, Train loss, err_ratio=(inf,0.328), Test loss, err_ratio=(15357.665,0.325)
prop=0.9, deg=3, g=0.000, l=0.000, Train loss, err_ratio=(inf,0.327), Test loss, err_ratio=(15430.190,0.324)
prop=0.9, deg=3, g=0.000, l=0.000, Train loss, err_ratio=(inf,0.328), Test loss, err_ratio=(15327.870,0.323)
prop=0.9, deg=3, g=0.000, l=0.001, Train loss, err_ratio=(inf,0.326), Test loss, err_ratio=(15427.238,0.327)
prop=0.9, deg=3, g=0.000, l=0.001, Train loss, err_ratio=(inf,0.327), Test loss, err_ratio=(15521.911,0.327)
prop=0.9, deg=3, g=0.000, l=0.004, Train loss, err_ratio=(inf,0.327), Test loss, err_ratio=(15445.791,0.327)
prop=0.9, deg=3, g=0.000, l=0.010, Train loss, err_ratio=(inf,0.327), Test loss, err_ratio=(15443.043,0.327)
prop=0.9, deg=3, g=0.000

prop=0.9, deg=3, g=0.500, l=0.000, Train loss, err_ratio=(inf,0.261), Test loss, err_ratio=(inf,0.265)
prop=0.9, deg=3, g=0.500, l=0.000, Train loss, err_ratio=(inf,0.287), Test loss, err_ratio=(inf,0.289)
prop=0.9, deg=3, g=0.500, l=0.001, Train loss, err_ratio=(inf,0.290), Test loss, err_ratio=(inf,0.292)
prop=0.9, deg=3, g=0.500, l=0.001, Train loss, err_ratio=(inf,0.281), Test loss, err_ratio=(inf,0.278)
prop=0.9, deg=3, g=0.500, l=0.004, Train loss, err_ratio=(inf,0.301), Test loss, err_ratio=(inf,0.301)
prop=0.9, deg=3, g=0.500, l=0.010, Train loss, err_ratio=(inf,0.281), Test loss, err_ratio=(inf,0.279)
prop=0.9, deg=3, g=0.500, l=0.027, Train loss, err_ratio=(inf,0.338), Test loss, err_ratio=(inf,0.340)
prop=0.9, deg=3, g=0.500, l=0.072, Train loss, err_ratio=(inf,0.333), Test loss, err_ratio=(inf,0.334)
prop=0.9, deg=3, g=0.500, l=0.193, Train loss, err_ratio=(inf,0.294), Test loss, err_ratio=(inf,0.297)
prop=0.9, deg=3, g=0.500, l=0.518, Train loss, err_ratio=(inf,0.339), Tes

prop=0.9, deg=7, g=0.100, l=0.010, Train loss, err_ratio=(inf,0.471), Test loss, err_ratio=(inf,0.472)
prop=0.9, deg=7, g=0.100, l=0.027, Train loss, err_ratio=(inf,0.494), Test loss, err_ratio=(inf,0.497)
prop=0.9, deg=7, g=0.100, l=0.072, Train loss, err_ratio=(inf,0.488), Test loss, err_ratio=(inf,0.487)
prop=0.9, deg=7, g=0.100, l=0.193, Train loss, err_ratio=(inf,0.478), Test loss, err_ratio=(inf,0.473)
prop=0.9, deg=7, g=0.100, l=0.518, Train loss, err_ratio=(inf,0.439), Test loss, err_ratio=(inf,0.441)
prop=0.9, deg=7, g=0.100, l=1.389, Train loss, err_ratio=(inf,0.513), Test loss, err_ratio=(inf,0.515)
prop=0.9, deg=7, g=0.100, l=3.728, Train loss, err_ratio=(inf,0.515), Test loss, err_ratio=(inf,0.520)
prop=0.9, deg=7, g=0.100, l=10.000, Train loss, err_ratio=(inf,0.549), Test loss, err_ratio=(inf,0.548)
prop=0.9, deg=7, g=0.500, l=0.000, Train loss, err_ratio=(inf,0.495), Test loss, err_ratio=(inf,0.490)
prop=0.9, deg=7, g=0.500, l=0.000, Train loss, err_ratio=(inf,0.446), Te

prop=0.9, deg=12, g=0.001, l=0.518, Train loss, err_ratio=(inf,0.603), Test loss, err_ratio=(inf,0.603)
prop=0.9, deg=12, g=0.001, l=1.389, Train loss, err_ratio=(inf,0.575), Test loss, err_ratio=(inf,0.574)
prop=0.9, deg=12, g=0.001, l=3.728, Train loss, err_ratio=(inf,0.559), Test loss, err_ratio=(inf,0.556)
prop=0.9, deg=12, g=0.001, l=10.000, Train loss, err_ratio=(inf,0.593), Test loss, err_ratio=(inf,0.594)
prop=0.9, deg=12, g=0.100, l=0.000, Train loss, err_ratio=(inf,0.601), Test loss, err_ratio=(inf,0.597)
prop=0.9, deg=12, g=0.100, l=0.000, Train loss, err_ratio=(inf,0.598), Test loss, err_ratio=(inf,0.596)
prop=0.9, deg=12, g=0.100, l=0.000, Train loss, err_ratio=(inf,0.602), Test loss, err_ratio=(inf,0.606)
prop=0.9, deg=12, g=0.100, l=0.000, Train loss, err_ratio=(inf,0.598), Test loss, err_ratio=(inf,0.599)
prop=0.9, deg=12, g=0.100, l=0.001, Train loss, err_ratio=(inf,0.604), Test loss, err_ratio=(inf,0.604)
prop=0.9, deg=12, g=0.100, l=0.001, Train loss, err_ratio=(inf,

KeyboardInterrupt: 

In [18]:
w_reg_log_reg = ws[np.argmin(ratio_err_tests)]
print(np.argmin(ratio_err_tests))
print(ratio_err_tests[np.argmin(ratio_err_tests)])
#print(w_reg_log_reg.shape)
print(len(ratio_err_tests))

116
0.21364
368


In [12]:
z = [1, 0, 2, 3]
print(np.argmin(z))

1


# Perform estimation on evaluation set
Une fois qu'on a le bon vecteur de poids, il suffit de runner ce code pour avoir le fichier de sortie.

In [92]:
ids_to_write = np.array([])
y_to_write = np.array([])

for r in range(len(x_ukn)):
    tx_ukn[r] = build_poly(x_ukn[r], 3)
    w_selected = w_LR_CV[r]
    y_ukn[r] = predict_labels(w_selected, tx_ukn[r], logistic_reg=True)
    y_ukn[r][y_ukn[r]==0] = -1
    
    ids_to_write = np.concatenate([ids_to_write, ids_ukn[r]], axis=0)
    y_to_write = np.concatenate([y_to_write, y_ukn[r]], axis=0)
    
create_csv_submission(ids_to_write, y_to_write, 'submission.csv')

568238


In [25]:
tx = build_poly(x, 11)
y_pred = predict_labels(w_LS_degree_CV, tx, logistic_reg=False)
y_pred[y_pred==0] = -1
create_csv_submission(ids, y_pred, 'nosubmission.csv')

In [88]:
a = np.array([1.0, 2.0])
b = np.array([3.0])
np.concatenate([a, b], axis = 0)

array([ 1.,  2.,  3.])